In [ ]:
pip install GPy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 959.4/959.4 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.3/71.3 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for GPy: filename=GPy-1.10.0-cp310-cp310-linux_x86_64.whl size=3211219 sha256=cac51c51b78527ca26ea6a6967da8a1ccae31e6a6f8867a4225b772883fc33fa
  Stored in directory: /root/.cache/pip/wheels/27/bd/9f/82ab4216eae088cba864ca0dc1d75699bd4bf6823790fb2f77
  Created wheel for paramz: filename=paramz-0.9.5-py3-none-any.whl size=102548 sha256=8e4952d2e0d932f1c39c827b2c7593e39cfbf4e84366020240c0666f78eacac1
  Stored in directory: /root/.cache/pip/wheels/d2/ef/9c/da9ceef7d0ff5287c24365844fc394852c2b79ac3fcf33bf8b
Successfully built GPy paramz


In [ ]:
pip install emukit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 kB 5.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.8 MB/s eta 0:00:00
  Created wheel for emukit: filename=emukit-0.4.10-py3-none-any.whl size=257055 sha256=271c92debf6ee6f202c1534ca43e5f4518fe568dc1af8b55fbfa70813a095ad3
  Stored in directory: /root/.cache/pip/wheels/30/d1/af/64ddede19b5eb77bd70362edebf9d393bf77dc34d354c11292
Successfully built emukit


In [ ]:
pip install pyDOE

  Preparing metadata (setup.py) ... done
  Created wheel for pyDOE: filename=pyDOE-0.3.8-py3-none-any.whl size=18167 sha256=7500f85f50a5ed0ca15b78e8c610b8e46fffbd231f646465cc93dd63d8028237
  Stored in directory: /root/.cache/pip/wheels/ce/b6/d7/c6b64746dba6433c593e471e0ac3acf4f36040456d1d160d17
Successfully built pyDOE


In [ ]:
import GPy
import numpy as np
from pyDOE import lhs


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd


ALL_xdata=pd.read_csv('/content/drive/MyDrive/ColabNotebooks/scaled_L2_4_dns.csv', header=None)

DNS_ydata=pd.read_csv('/content/drive/MyDrive/ColabNotebooks/scaled_wss_dns11_L2t4data.csv', header=None)
RANS_ydata=pd.read_csv('/content/drive/MyDrive/ColabNotebooks/scaled_wss_LCrans_all_L2t4data.csv',header=None)

Mounted at /content/drive


In [ ]:
# Generate sample data

x_train_l =  np.array([[70,100],[70,80],[70,50],[70,30],[70,0],[60,100],[60,80],[60,50],[60,30],[60,0],[50,100],[50,80],[50,50],[50,30],[50,0],[40,100],[40,80],[40,50],[40,30],[40,0]])
x_train_h = np.array([[70,100],[70,0],[60,80],[50,50],[40,100],[40,0]])

DNS_full=DNS_ydata.dropna(axis=1)
DNS_full=DNS_full.values
DNS=DNS_full[:,[0,2,3,7,8,10]]


DNS_test=DNS_full[:,[1,4,5,6,9]]
DNS_test=np.transpose(DNS_test)
RANS=RANS_ydata.dropna(axis=1)
RANS=RANS.values
RANS_full=RANS

RANS_test=RANS_full[:,[0,1,3,4,5,6,7,10,12,13,14,15,16,17,19]]



In [ ]:
# Max value of the WSS

# Find the maximum value in each column
RANS= np.max(RANS, axis=0)

RANS_full=np.max(RANS, axis=0)

RANS_test=np.max(RANS_test, axis=0)

DNS= np.max(DNS, axis=0)

DNS_full=np.max(DNS_full, axis=0)

DNS_test=np.max(DNS_test, axis=0)

In [ ]:
y_train_h=np.transpose(DNS)
y_train_l=np.transpose(RANS)

y_train_h=y_train_h.reshape(-1,1)
y_train_l=y_train_l.reshape(-1,1)

print(y_train_h)
print(y_train_l)

[[0.25852153]
 [0.19900103]
 [0.15969082]
 [0.09498576]
 [0.07385848]
 [0.05169693]]
[[0.18581525]
 [0.17990615]
 [0.17183543]
 [0.16698445]
 [0.16057095]
 [0.12863217]
 [0.12353837]
 [0.1166713 ]
 [0.11264634]
 [0.1073811 ]
 [0.08916878]
 [0.08496875]
 [0.07931362]
 [0.07604327]
 [0.07185387]
 [0.06082008]
 [0.05802784]
 [0.05313757]
 [0.05056323]
 [0.04735669]]


In [ ]:
# x_train_l=np.c_[ x_train_l, np.zeros(len(x_train_l)) ]
# x_train_h=np.c_[ x_train_h, np.ones(len(x_train_h)) ]
# X_train=np.append(x_train_l,x_train_h,axis=0)
#
from emukit.multi_fidelity.convert_lists_to_array import convert_x_list_to_array, convert_xy_lists_to_arrays

# Y_train=np.append(y_train_h,y_train_l,axis=0)
X_train, Y_train = convert_xy_lists_to_arrays([x_train_l, x_train_h], [y_train_l, y_train_h])
print(X_train.shape)
print(Y_train.shape)

print(X_train)

(26, 3)
(26, 1)
[[ 70. 100.   0.]
 [ 70.  80.   0.]
 [ 70.  50.   0.]
 [ 70.  30.   0.]
 [ 70.   0.   0.]
 [ 60. 100.   0.]
 [ 60.  80.   0.]
 [ 60.  50.   0.]
 [ 60.  30.   0.]
 [ 60.   0.   0.]
 [ 50. 100.   0.]
 [ 50.  80.   0.]
 [ 50.  50.   0.]
 [ 50.  30.   0.]
 [ 50.   0.   0.]
 [ 40. 100.   0.]
 [ 40.  80.   0.]
 [ 40.  50.   0.]
 [ 40.  30.   0.]
 [ 40.   0.   0.]
 [ 70. 100.   1.]
 [ 70.   0.   1.]
 [ 60.  80.   1.]
 [ 50.  50.   1.]
 [ 40. 100.   1.]
 [ 40.   0.   1.]]


In [ ]:
import GPy
import emukit.multi_fidelity
import emukit.test_functions
from emukit.model_wrappers.gpy_model_wrappers import GPyMultiOutputWrapper
from emukit.multi_fidelity.models import GPyLinearMultiFidelityModel
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
colors = dict(mcolors.BASE_COLORS, **mcolors.CSS4_COLORS)
%matplotlib inline

In [ ]:
## Create standard GP model using only high-fidelity data
x_train_DNS=x_train_h
y_train_DNS=y_train_h
kernelDNS = GPy.kern.ExpQuad(input_dim=2,lengthscale=[10,5.0],variance=0.0001, ARD=True)
kernelDNS.lengthscale.constrain_bounded(0,10000)
kernelDNS.variance.constrain_bounded(0,10)
high_gp_model = GPy.models.GPRegression(x_train_DNS, y_train_DNS, kernelDNS)
high_gp_model.Gaussian_noise.unfix()

## Fit the GP model

high_gp_model.optimize_restarts(100)

Optimization restart 1/100, f = -5.1789888258504035
Optimization restart 2/100, f = -6.239312839404395
Optimization restart 3/100, f = -9.454167359206668
Optimization restart 4/100, f = -9.454167359207096
Optimization restart 5/100, f = -6.007874114159641


 /usr/local/lib/python3.10/dist-packages/GPy/kern/src/stationary.py:243: RuntimeWarning:invalid value encountered in true_divide


Optimization restart 6/100, f = -6.146619193123065
Optimization restart 7/100, f = -9.454167355289105
Optimization restart 8/100, f = -8.648817027775747
Optimization restart 9/100, f = -8.648771931406365
Optimization restart 10/100, f = -6.041585255101165
Optimization restart 11/100, f = -5.955325604961654
Optimization restart 12/100, f = -8.648772110733274
Optimization restart 13/100, f = -9.454167359077966
Optimization restart 14/100, f = -9.454167359206403
Optimization restart 15/100, f = -8.648772068499818
Optimization restart 16/100, f = -9.454167358218672
Optimization restart 17/100, f = -6.109410291435361
Optimization restart 18/100, f = -9.45416735916714
Optimization restart 19/100, f = -9.454167359209995
Optimization restart 20/100, f = -6.152849750110472
Optimization restart 21/100, f = -9.45416735916947
Optimization restart 22/100, f = -5.856894281180229
Optimization restart 23/100, f = -6.143041120294538
Optimization restart 24/100, f = -6.0989809740812575
Optimization rest

In [ ]:
print(high_gp_model.ExpQuad.lengthscale[0])
print(high_gp_model.ExpQuad.lengthscale[1])
print(high_gp_model.ExpQuad.variance)
print(high_gp_model.gradient)
# Get the gradients of optimization
gradients_dns = high_gp_model.gradient

# Get the names of the model's parameters
parameter_dns = high_gp_model.parameter_names()

# Print the gradients along with the parameter names
print("Gradients of Optimization:")
for i in range(len(parameter_dns)):
    print(f"Parameter: {parameter_dns[i]} - Gradient: {gradients_dns[i]}")

38.57035543951553
593.2335157067455
  index  |  GP_regression.ExpQuad.variance  |  constraints  |  priors
  [0]    |                      0.06771242  |   0.0,10.0    |        
[ 4.68529303e-06 -2.16567311e-08 -6.72674472e-11  1.18611788e-03]
Gradients of Optimization:
Parameter: ExpQuad.variance - Gradient: 4.685293032550606e-06
Parameter: ExpQuad.lengthscale - Gradient: -2.16567311222235e-08
Parameter: Gaussian_noise.variance - Gradient: -6.726744715156144e-11


In [ ]:
## Create standard GP model using only low-fidelity data

x_train_RANS= np.array([[70,100],[70,80],[70,30],[70,0],[60,100],[60,80],[60,50],[50,100],[50,50],[50,30],[50,0],[40,100],[40,80],[40,50],[40,0]])

y_train_RANS=np.transpose(RANS_test)
y_train_RANS=y_train_RANS.reshape(-1,1)
print(y_train_RANS.shape)

(15, 1)


In [ ]:
kernelRANS = GPy.kern.ExpQuad(input_dim=2,lengthscale=[10,10],variance=0.05, ARD=True)
kernelRANS.lengthscale.constrain_bounded(0,1000)
kernelRANS.variance.constrain_bounded(0,10)
low_gp_model = GPy.models.GPRegression(x_train_RANS, y_train_RANS, kernelRANS)
low_gp_model.Gaussian_noise.unfix()

## Fit the GP model

low_gp_model.optimize_restarts(500)

Optimization restart 1/500, f = -69.36638157764469
Optimization restart 2/500, f = -44.37132871017219
Optimization restart 3/500, f = -53.42194388165345
Optimization restart 4/500, f = -69.36638158262683
Optimization restart 5/500, f = -69.36638158157464
Optimization restart 6/500, f = -69.36638158043993
Optimization restart 7/500, f = -69.36638158121374
Optimization restart 8/500, f = -69.3663815794193
Optimization restart 9/500, f = -69.36638158292033
Optimization restart 10/500, f = -69.23307760740127
Optimization restart 11/500, f = -69.36638158230663
Optimization restart 12/500, f = -67.13786383682867
Optimization restart 13/500, f = -45.96791824472188
Optimization restart 14/500, f = -67.13786393991337
Optimization restart 15/500, f = -50.65356522250316
Optimization restart 16/500, f = -67.1378638149675
Optimization restart 17/500, f = -69.3663815795669
Optimization restart 18/500, f = -45.995366621432694
Optimization restart 19/500, f = -69.36638158287681
Optimization restart 20

In [ ]:

#x_plot=np.reshape(x,[10000,2])
x_plot=np.array([[50,50]])
hf_mean_high_gp_model, hf_var_high_gp_model  = high_gp_model.predict(x_plot)


In [ ]:
print(low_gp_model.ExpQuad.lengthscale[0])
print(low_gp_model.ExpQuad.lengthscale[1])
print(low_gp_model.ExpQuad.variance)

# Get the gradients of optimization
gradients_rans = low_gp_model.gradient

# Get the names of the model's parameters
parameter_rans = low_gp_model.parameter_names()

# Print the gradients along with the parameter names
print("Gradients of Optimization:")
for i in range(len(parameter_rans)):
    print(f"Parameter: {parameter_rans[i]} - Gradient: {gradients_rans[i]}")

49.10220301119912
811.8406234938873
  index  |  GP_regression.ExpQuad.variance  |  constraints  |  priors
  [0]    |                      0.10458854  |   0.0,10.0    |        
Gradients of Optimization:
Parameter: ExpQuad.variance - Gradient: -7.188070067080596e-05
Parameter: ExpQuad.lengthscale - Gradient: 1.9434759587428038e-07
Parameter: Gaussian_noise.variance - Gradient: 5.351444413478173e-09


In [ ]:
kernels = [GPy.kern.ExpQuad(input_dim=2,lengthscale=[10,10],variance=0.005, ARD=True),GPy.kern.ExpQuad(input_dim=2,lengthscale=[10,10],variance=0.005, ARD=True)]
# for kernel in kernels:
#     kernel.lengthscale.constrain_bounded(0, 50)
#     kernel.variance.constrain_bounded(0, 10)
for i, kernel in enumerate(kernels):
    if i == 0:  # First fidelity
        kernel.lengthscale.constrain_bounded(0, 25)
        kernel.variance.constrain_bounded(0, 5)
    elif i == 1:  # Second fidelity
        kernel.lengthscale.constrain_bounded(0, 25)
        kernel.variance.constrain_bounded(0, 10)
lin_mf_kernel = emukit.multi_fidelity.kernels.LinearMultiFidelityKernel(kernels)

gpy_lin_mf_model = GPyLinearMultiFidelityModel(X_train, Y_train, lin_mf_kernel, n_fidelities=2)

# gpy_lin_mf_model.mixed_noise.Gaussian_noise.unfix()
# gpy_lin_mf_model.mixed_noise.Gaussian_noise_1.fix(0.001)

lin_mf_model =model=GPyMultiOutputWrapper(gpy_lin_mf_model, 2, n_optimization_restarts=50)

## Fit the model

lin_mf_model.optimize()

Optimization restart 1/50, f = -66.06113745808725
Optimization restart 2/50, f = -66.1562829279385
Optimization restart 3/50, f = -66.06095867250886
Optimization restart 4/50, f = -66.15625122660501
Optimization restart 5/50, f = -55.31534719878951
Optimization restart 6/50, f = -66.15628262716932
Optimization restart 7/50, f = -66.06095955875183
Optimization restart 8/50, f = -66.06095885921786
Optimization restart 9/50, f = -66.06095887459219
Optimization restart 10/50, f = -66.1447122517612
Optimization restart 11/50, f = -66.15628178908736
Optimization restart 12/50, f = -66.14471095694016
Optimization restart 13/50, f = -66.06096932500242
Optimization restart 14/50, f = -66.08545046931543
Optimization restart 15/50, f = -66.06095890949472
Optimization restart 16/50, f = -66.06094262939543
Optimization restart 17/50, f = -66.0609594031958
Optimization restart 18/50, f = -66.06100958491521
Optimization restart 19/50, f = -66.15627861112972
Optimization restart 20/50, f = -66.1447124

In [ ]:
 print(gpy_lin_mf_model)


Name : gp
Objective : -66.1562829279385
Number of Parameters : 9
Number of Optimization Parameters : 9
Updates : True
Parameters:
  gp.                                    |                   value  |  constraints  |  priors
  multifidelity.ExpQuad.variance         |    0.009288263115194885  |    0.0,5.0    |        
  multifidelity.ExpQuad.lengthscale      |                    (2,)  |   0.0,25.0    |        
  multifidelity.ExpQuad_1.variance       |   9.313601112639668e-05  |   0.0,10.0    |        
  multifidelity.ExpQuad_1.lengthscale    |                    (2,)  |   0.0,25.0    |        
  multifidelity.scale                    |      1.3170887349967952  |               |        
  mixed_noise.Gaussian_noise.variance    |  1.2824661906288589e-34  |      +ve      |        
  mixed_noise.Gaussian_noise_1.variance  |   1.305014231326087e-05  |      +ve      |        


In [ ]:
  print(gpy_lin_mf_model.multifidelity.ExpQuad.variance)
  print(gpy_lin_mf_model.multifidelity.ExpQuad_1.variance)
  print(gpy_lin_mf_model.multifidelity.ExpQuad.lengthscale)
  print(gpy_lin_mf_model.multifidelity.ExpQuad_1.lengthscale)

  index  |  gp.multifidelity.ExpQuad.variance  |  constraints  |  priors
  [0]    |                         0.00928826  |    0.0,5.0    |        
  index  |  gp.multifidelity.ExpQuad_1.variance  |  constraints  |  priors
  [0]    |                           0.00009314  |   0.0,10.0    |        
  index  |  gp.multifidelity.ExpQuad.lengthscale  |  constraints  |  priors
  [0]    |                           25.00000000  |   0.0,25.0    |        
  [1]    |                           25.00000000  |   0.0,25.0    |        
  index  |  gp.multifidelity.ExpQuad_1.lengthscale  |  constraints  |  priors
  [0]    |                             19.72484938  |   0.0,25.0    |        
  [1]    |                             25.00000000  |   0.0,25.0    |        


In [ ]:
# Get the gradients of optimization
gradients_mf = gpy_lin_mf_model.multifidelity.gradient

# Get the names of the model's parameters
parameter_mf = gpy_lin_mf_model.multifidelity.parameter_names()

# Print the gradients along with the parameter names
print("Gradients of Optimization:")
for i in range(len(parameter_mf)):
    print(f"Parameter: {parameter_mf[i]} - Gradient: {gradients_mf[i]}")

Gradients of Optimization:
Parameter: ExpQuad.variance - Gradient: 0.02830577606299255
Parameter: ExpQuad.lengthscale - Gradient: 0.30033683907368114
Parameter: ExpQuad_1.variance - Gradient: 0.7837477068817188
Parameter: ExpQuad_1.lengthscale - Gradient: 1.0603192797463976
Parameter: scale - Gradient: -4.7515151487257083e-07


In [ ]:
x_truth =  np.array([[70,100],[70,50],[70,0],[60,80],[60,30],[60,0],[50,80],[50,50],[40,100],[40,30],[40,0]])
y_truth=DNS_full.reshape(-1,1)
print(y_truth.shape)

(11, 1)


In [ ]:

truth_gp_model = GPy.models.GPRegression(x_truth, y_truth, kernelDNS)
truth_gp_model.Gaussian_noise.unfix()

## Fit the GP model

truth_gp_model.optimize_restarts(100)

Optimization restart 1/100, f = -35.394801992526396
Optimization restart 2/100, f = -35.39342397312936
Optimization restart 3/100, f = -35.3948019948095
Optimization restart 4/100, f = -35.394801992974756
Optimization restart 5/100, f = -35.3948019847341
Optimization restart 6/100, f = -20.915085704895215
Optimization restart 7/100, f = -35.394801986956836
Optimization restart 8/100, f = -11.947181342398661
Optimization restart 9/100, f = -35.39480199030487
Optimization restart 10/100, f = -35.39480199174985
Optimization restart 11/100, f = -22.863586217595365
Optimization restart 12/100, f = -28.007562753643626
Optimization restart 13/100, f = -35.393444369304596
Optimization restart 14/100, f = -13.98280549029171
Optimization restart 15/100, f = -35.394801994928045
Optimization restart 16/100, f = -35.39480199340399
Optimization restart 17/100, f = -35.39480199700404
Optimization restart 18/100, f = -22.687894347306646
Optimization restart 19/100, f = -35.39342397298973
Optimization 

In [ ]:

#x_plot=np.reshape(x,[10000,2])
x_plot=np.array([[70,50],[60,30],[60,0],[50,80],[40,30]])
X_plot_l=np.c_[ x_plot, np.zeros(len(x_plot)) ]
X_plot_h=np.c_[ x_plot, np.ones(len(x_plot)) ]

In [ ]:
hf_mean_lin_mf_model, hf_var_lin_mf_model = lin_mf_model.predict(X_plot_h)

hf_std_lin_mf_model = np.sqrt(hf_var_lin_mf_model)
hf_mean_high_gp_model, hf_var_high_gp_model  = high_gp_model.predict(X_plot_h)
lf_mean_low_gp_model, lf_var_low_gp_model  = low_gp_model.predict(X_plot_h)
tr_mean_truth_gp_model,tr_var_truth_gp_model=truth_gp_model.predict(X_plot_h)

In [ ]:
pip install chaospy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 12.6 MB/s eta 0:00:00


In [ ]:
import GPy
import chaospy as cp
import numpy as np


In [ ]:
import numpy as np

def saltelli_indices(model, X, n_samples, n_params, ranges):
    """
    Perform Monte Carlo estimation of Sobol' indices using Saltelli scheme.

    Parameters:
        model (callable): Function that represents the model.
        X (ndarray): Array of input parameter values.
        n_samples (int): Number of samples to generate.
        n_params (int): Number of parameters.
        ranges (list): List of tuples specifying the ranges for each parameter.

    Returns:
        S (ndarray): First-order Sobol' indices.
        ST (ndarray): Total-effect Sobol' indices.
    """
    # Generate uniform samples within the specified ranges
    samples = np.zeros((n_samples, n_params))
    for j in range(n_params):
        samples[:, j] = np.random.uniform(ranges[j][0], ranges[j][1], n_samples)

    # Scale the input samples to the desired range
    input_samples_h = np.hstack((samples, np.ones((n_samples, 1))))
    print(input_samples_h)
    # Initialize arrays for model evaluations
    Y = np.zeros(n_samples)
    Ys = np.zeros((n_samples, n_params))

    # Evaluate model at sample points
    out_m, out_v = model(X, input_samples_h)
    Y = out_m[:, -1]  # Extract the last column from out_m

    for j in range(n_params):
        input_samples_h[:, j] = ranges[j][1] - (input_samples_h[:, j] - ranges[j][0])
        out_m, out_v = model(X, input_samples_h)
        Ys[:, j] = out_m[:, -1]  # Extract the last column from out_m
        input_samples_h[:, j] = ranges[j][1] - (input_samples_h[:, j] - ranges[j][0])

    # Calculate mean of model evaluations
    Y_mean = np.mean(Y)

    # Initialize arrays for indices
    S = np.zeros(n_params)
    ST = np.zeros(n_params)

    # Calculate indices
    for j in range(n_params):
        S[j] = np.mean(Y * (Ys[:, j] - Y_mean))

        # Calculate the normalization factor for the total-effect Sobol' indices
        norm_factor = 1 / np.var(Y)
        ST[j] = norm_factor * (np.mean((Y - Ys[:, j])**2) / np.var(Y))

    return S, ST

# Example usage
def model(X, params):
    # Example model function
    out_m = np.zeros((params.shape[0], X.shape[0]+1))
    out_v = np.zeros(params.shape[0])

    # Your model code here...
    out_m, out_v = lin_mf_model.predict(params)
    return out_m, out_v

# Define input parameter values
X = np.array([1, 2])

# Define number of samples and parameters
n_samples = 10000000
n_params = 2

# Define ranges for each parameter
ranges = [(40, 70), (0, 100)]

# Perform Monte Carlo estimation of Sobol' indices
S, ST = saltelli_indices(model, X, n_samples, n_params, ranges)

# Normalize the total-effect Sobol' indices
ST /= np.sum(ST)


print("Total-effect Sobol' indices:", ST)
print('MF-Prediction')

[[59.19930291 14.24778824  1.        ]
 [68.93859156 95.75040717  1.        ]
 [48.82997285 93.18689905  1.        ]
 ...
 [59.64630395 77.59128026  1.        ]
 [66.23598898 55.60085795  1.        ]
 [55.9785021  74.31475028  1.        ]]
Total-effect Sobol' indices: [0.94503489 0.05496511]
MF-Prediction


In [ ]:
import numpy as np

def saltelli_indices(model, X, n_samples, n_params, ranges):
    """
    Perform Monte Carlo estimation of Sobol' indices using Saltelli scheme.

    Parameters:
        model (callable): Function that represents the model.
        X (ndarray): Array of input parameter values.
        n_samples (int): Number of samples to generate.
        n_params (int): Number of parameters.
        ranges (list): List of tuples specifying the ranges for each parameter.

    Returns:
        S (ndarray): First-order Sobol' indices.
        ST (ndarray): Total-effect Sobol' indices.
    """
    # Generate uniform samples within the specified ranges
    samples = np.zeros((n_samples, n_params))
    for j in range(n_params):
        samples[:, j] = np.random.uniform(ranges[j][0], ranges[j][1], n_samples)

    # Scale the input samples to the desired range
    input_samples_h = np.hstack((samples, np.zeros((n_samples, 1))))
    print(input_samples_h)
    # Initialize arrays for model evaluations
    Y = np.zeros(n_samples)
    Ys = np.zeros((n_samples, n_params))

    # Evaluate model at sample points
    out_m, out_v = model(X, input_samples_h)
    Y = out_m[:, -1]  # Extract the last column from out_m

    for j in range(n_params):
        input_samples_h[:, j] = ranges[j][1] - (input_samples_h[:, j] - ranges[j][0])
        out_m, out_v = model(X, input_samples_h)
        Ys[:, j] = out_m[:, -1]  # Extract the last column from out_m
        input_samples_h[:, j] = ranges[j][1] - (input_samples_h[:, j] - ranges[j][0])

    # Calculate mean of model evaluations
    Y_mean = np.mean(Y)

    # Initialize arrays for indices
    S = np.zeros(n_params)
    ST = np.zeros(n_params)

    # Calculate indices
    for j in range(n_params):
        S[j] = np.mean(Y * (Ys[:, j] - Y_mean))

        # Calculate the normalization factor for the total-effect Sobol' indices
        norm_factor = 1 / np.var(Y)
        ST[j] = norm_factor * (np.mean((Y - Ys[:, j])**2) / np.var(Y))

    return S, ST

# Example usage
def model(X, params):
    # Example model function
    out_m = np.zeros((params.shape[0], X.shape[0]+1))
    out_v = np.zeros(params.shape[0])

    # Your model code here...
    out_m, out_v = high_gp_model.predict(params)
    return out_m, out_v

# Define input parameter values
X = np.array([1, 2])

# Define number of samples and parameters
n_samples = 10000000
n_params = 2

# Define ranges for each parameter
ranges = [(40, 70), (0, 100)]

# Perform Monte Carlo estimation of Sobol' indices
S, ST = saltelli_indices(model, X, n_samples, n_params, ranges)

# Normalize the total-effect Sobol' indices
ST /= np.sum(ST)

# Print the results
print("Total-effect Sobol' indices:", ST)
print('DNS-GPR')

[[48.53517822 38.60761169  0.        ]
 [43.00909224 11.72441126  0.        ]
 [43.4373133  20.52582052  0.        ]
 ...
 [69.55149676 17.29118835  0.        ]
 [52.94763538 20.23768956  0.        ]
 [56.4757557  94.74466601  0.        ]]
Total-effect Sobol' indices: [0.92343429 0.07656571]
DNS-GPR


In [ ]:
import numpy as np

def saltelli_indices(model, X, n_samples, n_params, ranges):
    """
    Perform Monte Carlo estimation of Sobol' indices using Saltelli scheme.

    Parameters:
        model (callable): Function that represents the model.
        X (ndarray): Array of input parameter values.
        n_samples (int): Number of samples to generate.
        n_params (int): Number of parameters.
        ranges (list): List of tuples specifying the ranges for each parameter.

    Returns:
        S (ndarray): First-order Sobol' indices.
        ST (ndarray): Total-effect Sobol' indices.
    """
    # Generate uniform samples within the specified ranges
    samples = np.zeros((n_samples, n_params))
    for j in range(n_params):
        samples[:, j] = np.random.uniform(ranges[j][0], ranges[j][1], n_samples)

    # Scale the input samples to the desired range
    input_samples_h = np.hstack((samples, np.zeros((n_samples, 1))))
    print(input_samples_h)
    # Initialize arrays for model evaluations
    Y = np.zeros(n_samples)
    Ys = np.zeros((n_samples, n_params))

    # Evaluate model at sample points
    out_m, out_v = model(X, input_samples_h)
    Y = out_m[:, -1]  # Extract the last column from out_m

    for j in range(n_params):
        input_samples_h[:, j] = ranges[j][1] - (input_samples_h[:, j] - ranges[j][0])
        out_m, out_v = model(X, input_samples_h)
        Ys[:, j] = out_m[:, -1]  # Extract the last column from out_m
        input_samples_h[:, j] = ranges[j][1] - (input_samples_h[:, j] - ranges[j][0])

    # Calculate mean of model evaluations
    Y_mean = np.mean(Y)

    # Initialize arrays for indices
    S = np.zeros(n_params)
    ST = np.zeros(n_params)

    # Calculate indices
    for j in range(n_params):
        S[j] = np.mean(Y * (Ys[:, j] - Y_mean))

        # Calculate the normalization factor for the total-effect Sobol' indices
        norm_factor = 1 / np.var(Y)
        ST[j] = norm_factor * (np.mean((Y - Ys[:, j])**2) / np.var(Y))

    return S, ST

# Example usage
def model(X, params):
    # Example model function
    out_m = np.zeros((params.shape[0], X.shape[0]+1))
    out_v = np.zeros(params.shape[0])

    # Your model code here...
    out_m, out_v =low_gp_model.predict(params)
    return out_m, out_v

# Define input parameter values
X = np.array([1, 2])

# Define number of samples and parameters
n_samples = 10000000
n_params = 2

# Define ranges for each parameter
ranges = [(40, 70), (0, 100)]

# Perform Monte Carlo estimation of Sobol' indices
S, ST = saltelli_indices(model, X, n_samples, n_params, ranges)

# Normalize the total-effect Sobol' indices
ST /= np.sum(ST)

# Print the results
print("Total-effect Sobol' indices:", ST)
print('RANS-GPR')

[[44.91534432 68.74509147  0.        ]
 [59.94578554 60.25933702  0.        ]
 [55.24979259 80.39186724  0.        ]
 ...
 [65.87832918 12.58148775  0.        ]
 [49.37990687 71.97108158  0.        ]
 [67.45839791 83.0925956   0.        ]]
Total-effect Sobol' indices: [0.97193607 0.02806393]
RANS-GPR


In [ ]:
import numpy as np

def saltelli_indices(model, X, n_samples, n_params, ranges):
    """
    Perform Monte Carlo estimation of Sobol' indices using Saltelli scheme.

    Parameters:
        model (callable): Function that represents the model.
        X (ndarray): Array of input parameter values.
        n_samples (int): Number of samples to generate.
        n_params (int): Number of parameters.
        ranges (list): List of tuples specifying the ranges for each parameter.

    Returns:
        S (ndarray): First-order Sobol' indices.
        ST (ndarray): Total-effect Sobol' indices.
    """
    # Generate uniform samples within the specified ranges
    samples = np.zeros((n_samples, n_params))
    for j in range(n_params):
        samples[:, j] = np.random.uniform(ranges[j][0], ranges[j][1], n_samples)

    # Scale the input samples to the desired range
    input_samples_h = np.hstack((samples, np.zeros((n_samples, 1))))
    print(input_samples_h)
    # Initialize arrays for model evaluations
    Y = np.zeros(n_samples)
    Ys = np.zeros((n_samples, n_params))

    # Evaluate model at sample points
    out_m, out_v = model(X, input_samples_h)
    Y = out_m[:, -1]  # Extract the last column from out_m

    for j in range(n_params):
        input_samples_h[:, j] = ranges[j][1] - (input_samples_h[:, j] - ranges[j][0])
        out_m, out_v = model(X, input_samples_h)
        Ys[:, j] = out_m[:, -1]  # Extract the last column from out_m
        input_samples_h[:, j] = ranges[j][1] - (input_samples_h[:, j] - ranges[j][0])

    # Calculate mean of model evaluations
    Y_mean = np.mean(Y)

    # Initialize arrays for indices
    S = np.zeros(n_params)
    ST = np.zeros(n_params)

    # Calculate indices
    for j in range(n_params):
        S[j] = np.mean(Y * (Ys[:, j] - Y_mean))

        # Calculate the normalization factor for the total-effect Sobol' indices
        norm_factor = 1 / np.var(Y)
        ST[j] = norm_factor * (np.mean((Y - Ys[:, j])**2) / np.var(Y))

    return S, ST

# Example usage
def model(X, params):
    # Example model function
    out_m = np.zeros((params.shape[0], X.shape[0]+1))
    out_v = np.zeros(params.shape[0])

    # Your model code here...
    out_m, out_v =truth_gp_model.predict(params)
    return out_m, out_v

# Define input parameter values
X = np.array([1, 2])

# Define number of samples and parameters
n_samples = 10000000
n_params = 2

# Define ranges for each parameter
ranges = [(40, 70), (0, 100)]

# Perform Monte Carlo estimation of Sobol' indices
S, ST = saltelli_indices(model, X, n_samples, n_params, ranges)

# Normalize the total-effect Sobol' indices
ST /= np.sum(ST)

# Print the results
print("Total-effect Sobol' indices:", ST)
print('Truth')

[[45.74549231 56.5011724   0.        ]
 [63.38475443 87.95537493  0.        ]
 [63.12896495 19.25125308  0.        ]
 ...
 [43.26063636  3.49460748  0.        ]
 [60.80100675 65.95896321  0.        ]
 [48.83350988 16.93223883  0.        ]]
Total-effect Sobol' indices: [0.94074353 0.05925647]
Truth
